<a href="https://colab.research.google.com/github/abdallaho98/AutoSpeech/blob/master/NB_and_Logistical_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 169kB/s 
     |████████████████████████████████| 421.8MB 24kB/s 
     |████████████████████████████████| 983kB 35.4MB/s 
     |████████████████████████████████| 245kB 44.7MB/s 
     |████████████████████████████████| 778kB 41.9MB/s 
     |████████████████████████████████| 471kB 39.5MB/s 
     |████████████████████████████████| 3.9MB 45.4MB/s 
     |████████████████████████████████| 450kB 42.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 3.0MB 40.5MB/s 
     |████████████████████████████████| 1.1MB 36.3MB/s 
     |████████████████████████████████| 890kB 43.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.19.8-cp36-none-any.whl size=25268462 sha256=615414c88feac965e094bf491f034c7db24ae0c06a768fe2677504f75b92e90d
  Stored in directory: /root/.cache/pip/wheels/53/0d/49/76e18efeba19337a3dd626260325525e61c2e83974d6bbeae4
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [ ]:
import ktrain
from ktrain import text

In [243]:
import pandas as pd
filepath = 'legalts.txt'
df = pd.DataFrame(columns=('rating', 'review'))
with open(filepath) as fp:
   line = fp.readline()
   cnt = 0
   while line:
       x = line.strip().split("  ")
       df.loc[cnt] = [int(x[1]) , x[0]]
       line = fp.readline()
       cnt += 1
df['rating'] = df['rating'].apply(lambda x: x)
df.head()

,rating,review
0,4,ليس للشركاء في مال شائع أن يطلبوا قسمته إذا تب...
1,5,إذا وقع تحريف في نص السفتجة فالأشخاص الموقعون ...
2,2,يجوز الحجز على الأجر أو المرتب بقيمة النفقة ال...
3,3,يعين القاضي طبقا لأحكام قانون الأسرة، مقدما من...
4,2,يجوز للعموم المعارضة في أي تصحيح قضائي أو إدار...


In [262]:
import pandas as pd
import re
import string
import snowballstemmer
ar_light_stem = snowballstemmer.stemmer('arabic') 
filepath = 'legalts.txt'
df = pd.DataFrame(columns=('rating', 'review'))
with open(filepath) as fp:
   line = fp.readline()
   cnt = 0
   while line:
       x = line.strip().split("  ")
       result = re.sub(r'[0-9,.()،]+', '', x[0])
       listStrin = [ar_light_stem.stemWord(text) for text in result.split(' ')]
       strin = ' '.join(listStrin)
       df.loc[cnt] = [int(x[1]) , strin]
       line = fp.readline()
       cnt += 1
df['rating'] = df['rating'].apply(lambda x: x)
df.head()

,rating,review
0,4,ليس شركاء في مال شايع ان يطلب قسم اذا تبين من ...
1,5,اذا وقع تحريف في نص سفتج فالاشخاص موقع علي بعد...
2,2,يجوز حجز علي اجر او مرتب قيم نفق غذاء اذا كان ...
3,3,يعين قاض طبق لاحكام قانون اسر مقدم من بين اقار...
4,2,يجوز عموم معارض في اي تصحيح قضاء او ادار لعقد ...


In [263]:
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [264]:
X_train, X_test, y_train, y_test = train_test_split(df['review'].values, 
                 df['rating'].values,        
                 test_size=0.2)

In [265]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [266]:
vect = CountVectorizer(tokenizer=tokenize)
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [267]:
model = LogisticRegression(C=3, dual=False)
model.fit(tf_train, y_train)
preds = model.predict(tf_test)
accuracy = (preds == y_test).mean()
print(accuracy)

0.7874396135265701


In [268]:
vect = TfidfVectorizer(strip_accents='unicode', tokenizer=tokenize, ngram_range=(1, 3), max_df=0.9, min_df=5, sublinear_tf=True)
tfidf_train = vect.fit_transform(X_train)
tfidf_test = vect.transform(X_test)
model = LogisticRegression(C=30,max_iter=500, dual=False)
model.fit(tfidf_train, y_train)
preds = model.predict(tfidf_test)
accuracy = (preds==y_test).mean()
print(accuracy)

0.7681159420289855


In [269]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB().fit(tfidf_train.todense(), y_train)
predicted_y = model.predict(tfidf_test.todense())
from sklearn.metrics import accuracy_score

# now calculating that how much accurate our model is with comparing our predicted values and y_test values
accuracy_score = accuracy_score(y_test, predicted_y) 
print (accuracy_score)

0.46859903381642515


In [272]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(tfidf_train, y_train)
mnb.score(tfidf_test, y_test)

0.5748792270531401

In [273]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(tf_train, y_train)
mnb.score(tf_test, y_test)

0.5845410628019324